<a href="https://colab.research.google.com/github/davidbro-in/natural-language-processing/blob/main/3_custom_embedding_using_gensim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://www.inove.com.ar"><img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center"></a>


# Procesamiento de lenguaje natural
## Custom embedddings con Gensim



### Objetivo
El objetivo es utilizar documentos / corpus para crear embeddings de palabras basado en ese contexto. Se utilizará canciones de bandas para generar los embeddings, es decir, que los vectores tendrán la forma en función de como esa banda haya utilizado las palabras en sus canciones.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import multiprocessing
from gensim.models import Word2Vec

### Datos
Utilizaremos como dataset el manifiesto comunista.

In [4]:
!wget https://www.gutenberg.org/cache/epub/31193/pg31193.txt

--2021-10-20 04:12:07--  https://www.gutenberg.org/cache/epub/31193/pg31193.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 114408 (112K) [text/plain]
Saving to: ‘pg31193.txt’

pg31193.txt         100%[===================>] 111.73K   394KB/s    in 0.3s    

2021-10-20 04:12:08 (394 KB/s) - ‘pg31193.txt’ saved [114408/114408]



In [5]:
# Armar el dataset utilizando salto de línea para separar las oraciones/docs
df = pd.read_csv('/content/pg31193.txt', sep='/n', header=None)
df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


,0
0,The Project Gutenberg EBook of Manifesto of th...
1,Karl Marx and Frederick Engels
2,This eBook is for the use of anyone anywhere a...
3,almost no restrictions whatsoever. You may co...
4,re-use it under the terms of the Project Guten...


In [6]:
print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 1796


### 1 - Preprocesamiento

In [7]:
from keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))

In [8]:
# Demos un vistazo
sentence_tokens[:2]

[['the',
  'project',
  'gutenberg',
  'ebook',
  'of',
  'manifesto',
  'of',
  'the',
  'communist',
  'party',
  'by'],
 ['karl', 'marx', 'and', 'frederick', 'engels']]

### 2 - Crear los vectores (word2vec)

In [9]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobracargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [10]:
# Crearmos el modelo generador de vectoeres
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     size=300,       # dimensionalidad de los vectores 
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

In [11]:
# Buildear el vocabularui con los tokens
w2v_model.build_vocab(sentence_tokens)

In [12]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 1796


In [13]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.vocab))

Cantidad de words distintas en el corpus: 523


### 3 - Entrenar el modelo generador

In [14]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=20,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 141801.171875
Loss after epoch 1: 74296.515625
Loss after epoch 2: 72341.53125
Loss after epoch 3: 72245.71875
Loss after epoch 4: 71782.25
Loss after epoch 5: 71966.21875
Loss after epoch 6: 72309.71875
Loss after epoch 7: 73580.25
Loss after epoch 8: 73048.8125
Loss after epoch 9: 73517.6875
Loss after epoch 10: 73399.75
Loss after epoch 11: 73346.1875
Loss after epoch 12: 72901.75
Loss after epoch 13: 67412.6875
Loss after epoch 14: 63934.125
Loss after epoch 15: 62974.75
Loss after epoch 16: 62865.0
Loss after epoch 17: 63617.75
Loss after epoch 18: 63221.25
Loss after epoch 19: 63571.75


(166408, 357940)

### 4 - Ensayar

In [15]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["capital"], topn=10)

[('national', 0.9929433465003967),
 ('become', 0.9923353791236877),
 ('wage', 0.9917056560516357),
 ('since', 0.9908077716827393),
 ('where', 0.9907176494598389),
 ('ever', 0.9903765916824341),
 ('english', 0.99002605676651),
 ('developed', 0.9900137782096863),
 ('because', 0.9897047877311707),
 ('ideas', 0.9895086884498596)]

In [16]:
# Palabras que MENOS se relacionan con...:
w2v_model.wv.most_similar(positive=["right"], topn=10)

[('little', 0.9990811944007874),
 ('help', 0.9986866116523743),
 ('express', 0.9986482858657837),
 ('written', 0.9986408948898315),
 ('money', 0.9983272552490234),
 ('say', 0.9983018636703491),
 ('except', 0.9982881546020508),
 ('given', 0.9982777833938599),
 ('less', 0.9982442855834961),
 ('either', 0.998201847076416)]

In [17]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["power"], topn=10)

[('order', 0.9991291165351868),
 ('supremacy', 0.9990858435630798),
 ('character', 0.9989020824432373),
 ('whose', 0.9987571835517883),
 ('made', 0.9985997080802917),
 ('then', 0.9984824061393738),
 ('great', 0.9984492063522339),
 ('life', 0.9981537461280823),
 ('material', 0.9980261921882629),
 ('england', 0.9979739189147949)]

In [18]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["state"], topn=5)

[('existing', 0.996415376663208),
 ('proletariat', 0.9960031509399414),
 ('classes', 0.9943127632141113),
 ('between', 0.9936330914497375),
 ('ruling', 0.9936048984527588)]

In [19]:
# Ensayar con una palabra que no está en el corpus (en vocab):
w2v_model.wv.most_similar(negative=["work"])

[('class', -0.30893924832344055),
 ('working', -0.3570108413696289),
 ('modern', -0.3726712763309479),
 ('conditions', -0.43376317620277405),
 ('has', -0.4401751160621643),
 ('against', -0.45149046182632446),
 ('production', -0.45636609196662903),
 ('bourgeois', -0.4832618832588196),
 ('industry', -0.4859587550163269),
 ('political', -0.49210095405578613)]

### 5 - Visualizar agrupación de vectores

In [20]:
from sklearn.decomposition import IncrementalPCA    
from sklearn.manifold import TSNE                   
import numpy as np                                  

def reduce_dimensions(model):
    num_dimensions = 2  

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index2word)  

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    x_vals = [v[0] for v in vectors]
    y_vals = [v[1] for v in vectors]
    return x_vals, y_vals, labels

In [21]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

x_vals, y_vals, labels = reduce_dimensions(w2v_model)

MAX_WORDS=200
fig = px.scatter(x=x_vals[:MAX_WORDS], y=y_vals[:MAX_WORDS], text=labels[:MAX_WORDS])
fig.show(renderer="colab") # esto para plotly en colab
